In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
!pip install tensorflow-addons
import tensorflow_addons as tfa
!pip install patool
import patoolib
import os


In [16]:

model  = tf.keras.models.load_model('/content/drive/MyDrive/model/mymodel')

In [3]:

patoolib.extract_archive('/content/test_images.rar')

patool: Extracting /content/test_images.rar ...
patool: running /usr/bin/unrar x -- /content/test_images.rar
patool:     with cwd='./Unpack_x_l6mz7w'
patool: ... /content/test_images.rar extracted to `test_images1' (local file exists).


'test_images1'

In [5]:
names = os.listdir('/content/test_images')
print(names)

['picture1.jpg', 'Test_images.txt', 'normalglasses.jpg', 'freakles_img_2.jpg', 'statute2.jpg', 'picture4.jpg', 'picture5.jpg', 'picture2.jpg', 'normalglasses2.jpg', 'freakles_img_1.jpg', 'sunglasses.jpg', 'statute.jpg', 'picture3.jpg']


In [17]:
df = pd.DataFrame((names),columns=['Image_Path'])

In [18]:
df.head()

,Image_Path
0,picture1.jpg
1,Test_images.txt
2,normalglasses.jpg
3,freakles_img_2.jpg
4,statute2.jpg


In [3]:
test_dataGen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)
test_gen = test_dataGen.flow_from_dataframe(df,directory='/content/test_images',x_col = 'Image_Path',y_col = None,Batch_size = 1,target_size=(512,512),class_mode=None)


NameError: name 'ImageDataGenerator' is not defined

In [20]:
pred = model.predict(test_gen,batch_size  = 16)

In [21]:
pred

array([[9.64312465e-04, 1.13980728e-03, 3.61273880e-04, 4.13601956e-04,
        2.50909041e-04, 1.76989182e-04, 1.14610601e-04, 5.96310892e-05,
        1.45367987e-04, 3.19312210e-04, 6.19555358e-04, 8.35015476e-01,
        1.12512862e-04, 9.05446450e-06, 1.59360607e-05, 3.64918560e-05,
        1.55395290e-04, 1.59985840e-01, 1.03910228e-04],
       [6.46051834e-04, 1.42007961e-03, 4.82864823e-04, 5.47461794e-04,
        2.29877012e-04, 1.97584959e-04, 1.60182768e-04, 8.24132003e-05,
        1.60406416e-04, 3.43540683e-04, 5.57393476e-04, 8.88641596e-01,
        2.25125710e-04, 9.09082337e-06, 3.18369966e-05, 2.43373779e-05,
        3.61072482e-04, 1.05757646e-01, 1.21424098e-04],
       [6.73413184e-03, 5.38781146e-03, 2.61641201e-03, 1.60989445e-03,
        1.12687168e-03, 1.22930505e-03, 3.33999493e-03, 7.96951412e-04,
        8.75805039e-04, 1.19134756e-02, 2.45003286e-03, 2.85494059e-01,
        4.70382534e-03, 2.06914730e-04, 2.78700667e-04, 7.18954951e-04,
        8.27121548e-04

In [22]:
preds =  []
for i in range(len(pred)):
  pr = []
  pr.append(df['Image_Path'].values[i])
  for j in range(len(pred[i])):

    #for wrinkles and freakles 
    if (j==0) or (j==1):
      if pred[i][j] >0.5:
          pr.append(1)
      else:
        pr.append(1)
    #for glasses 
    if j==2:
      z = np.array(pred[i][j:j+3])
      #print(z)
      #print(np.argmax(z))
  
      pr.append(np.argmax(z))
      
    #hair_colour
    if j==5:
      z = np.array(pred[i][j:j+9])    
      pr.append(np.argmax(z))

    if j==14:
      z = np.array(pred[i][j:])
      pr.append(np.argmax(z))

  preds.append(pr)

In [23]:
preds

[['picture1.jpg', 1, 1, 1, 6, 3],
 ['Test_images.txt', 1, 1, 1, 6, 3],
 ['normalglasses.jpg', 1, 1, 0, 6, 3],
 ['freakles_img_2.jpg', 1, 1, 0, 6, 3],
 ['statute2.jpg', 1, 1, 1, 6, 3],
 ['picture4.jpg', 1, 1, 1, 6, 3],
 ['picture5.jpg', 1, 1, 1, 6, 3],
 ['picture2.jpg', 1, 1, 1, 6, 3],
 ['normalglasses2.jpg', 1, 1, 1, 6, 3],
 ['freakles_img_1.jpg', 1, 1, 1, 6, 3],
 ['sunglasses.jpg', 1, 1, 1, 6, 3],
 ['statute.jpg', 1, 1, 0, 6, 3]]

In [2]:
predictions = pd.DataFrame(preds,columns = ['mage_name','wrinkles','freakles','glasses','hair_color','hair_top'])

NameError: name 'pd' is not defined

In [1]:
predictions.to_csv('annotations.csv')

NameError: name 'predictions' is not defined